# Crypto Data Analysis - ITMO MD 2025

Анализ данных криптовалютного рынка с использованием данных из Data Mart слоя.

**Источники данных:**
- `dm.dm_crypto_market_overview` - обзор рынка с ценами, MA, волатильностью
- `dm.dm_news_impact_analysis` - анализ влияния новостей на цены
- `ods.ods_binance_daily_agg` - дневные агрегаты цен
- `ods.ods_news_enriched` - обогащенные новости с sentiment analysis

In [ ]:
# Импорты
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from datetime import datetime, timedelta
import warnings

warnings.filterwarnings('ignore')

# Настройка стиля графиков
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline

# Размер фигур по умолчанию
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 11

## 1. Подключение к базе данных

In [ ]:
# Параметры подключения к PostgreSQL
conn_params = {
    'host': 'localhost',  # Или IP сервера
    'port': 5433,
    'database': 'analytics',
    'user': 'analytics',
    'password': 'analytics'
}

# Подключение
conn = psycopg2.connect(**conn_params)
print("✅ Подключение к PostgreSQL успешно установлено")

# Функция для выполнения SQL запросов
def query_to_df(sql):
    """Выполнить SQL запрос и вернуть DataFrame"""
    return pd.read_sql(sql, conn)

## 2. Загрузка данных из Data Mart

In [ ]:
# Загрузка данных обзора рынка
market_overview = query_to_df("""
    SELECT 
        symbol,
        date,
        current_price,
        ma_7d,
        ma_30d,
        volatility_7d,
        trend_indicator,
        total_volume
    FROM dm.dm_crypto_market_overview
    ORDER BY date DESC, symbol
""")

# Загрузка анализа влияния новостей
news_impact = query_to_df("""
    SELECT 
        date,
        total_news,
        positive_news,
        negative_news,
        neutral_news,
        avg_sentiment_score,
        avg_price_change_pct,
        sentiment_price_correlation
    FROM dm.dm_news_impact_analysis
    ORDER BY date DESC
""")

# Загрузка дневных агрегатов для детального анализа
daily_agg = query_to_df("""
    SELECT 
        symbol,
        date,
        open_price,
        close_price,
        high_price,
        low_price,
        total_volume,
        price_change_pct
    FROM ods.ods_binance_daily_agg
    ORDER BY date DESC, symbol
""")

# Загрузка новостей с sentiment
news_enriched = query_to_df("""
    SELECT 
        pub_date::date as date,
        title,
        sentiment,
        title_length,
        description_length
    FROM ods.ods_news_enriched
    ORDER BY pub_date DESC
    LIMIT 1000
""")

print(f"✅ Загружено:")
print(f"   - Market Overview: {len(market_overview)} строк")
print(f"   - News Impact: {len(news_impact)} строк")
print(f"   - Daily Aggregates: {len(daily_agg)} строк")
print(f"   - News Enriched: {len(news_enriched)} строк")

## 3. Обзор данных

In [ ]:
# Основная статистика по рынку
print("=== Статистика по символам ===")
print(market_overview.groupby('symbol')['current_price'].agg(['count', 'mean', 'min', 'max']))
print("\n=== Распределение трендов ===")
print(market_overview['trend_indicator'].value_counts())
print("\n=== Распределение sentiment новостей ===")
print(news_enriched['sentiment'].value_counts())

## 4. Визуализация 1: Динамика цен с Moving Averages

**Инсайт**: Визуализация показывает ценовую динамику BTC с наложением скользящих средних (7 и 30 дней), что позволяет идентифицировать краткосрочные и долгосрочные тренды.

In [ ]:
# Фильтр для BTCUSDT
btc_data = market_overview[market_overview['symbol'] == 'BTCUSDT'].sort_values('date')

if len(btc_data) > 0:
    fig, ax = plt.subplots(figsize=(16, 7))
    
    # Цена и скользящие средние
    ax.plot(btc_data['date'], btc_data['current_price'], 
            label='Current Price', linewidth=2, color='#1f77b4')
    ax.plot(btc_data['date'], btc_data['ma_7d'], 
            label='MA 7 days', linewidth=1.5, linestyle='--', color='#ff7f0e')
    ax.plot(btc_data['date'], btc_data['ma_30d'], 
            label='MA 30 days', linewidth=1.5, linestyle='--', color='#2ca02c')
    
    ax.set_xlabel('Date', fontsize=12, fontweight='bold')
    ax.set_ylabel('Price (USDT)', fontsize=12, fontweight='bold')
    ax.set_title('BTC/USDT Price Dynamics with Moving Averages', 
                 fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='best', fontsize=11)
    ax.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Инсайты
    print("\n📊 ИНСАЙТЫ:")
    latest = btc_data.iloc[-1]
    print(f"   - Текущая цена BTC: ${latest['current_price']:,.2f}")
    print(f"   - MA(7): ${latest['ma_7d']:,.2f}")
    print(f"   - MA(30): ${latest['ma_30d']:,.2f}")
    print(f"   - Текущий тренд: {latest['trend_indicator']}")
    
    if latest['current_price'] > latest['ma_7d'] > latest['ma_30d']:
        print("   ✅ Сильный восходящий тренд (цена > MA7 > MA30)")
    elif latest['current_price'] < latest['ma_7d'] < latest['ma_30d']:
        print("   ⚠️ Сильный нисходящий тренд (цена < MA7 < MA30)")
    else:
        print("   ⚡ Консолидация или разворот тренда")
else:
    print("⚠️ Нет данных для BTCUSDT")

## 5. Визуализация 2: Волатильность по символам

**Инсайт**: Box plot показывает распределение 7-дневной волатильности для разных криптовалют, позволяя сравнить риски инвестирования.

In [ ]:
if len(market_overview) > 0:
    fig, ax = plt.subplots(figsize=(14, 7))
    
    # Boxplot волатильности по символам
    market_overview.boxplot(column='volatility_7d', by='symbol', ax=ax, 
                            patch_artist=True, showmeans=True)
    
    ax.set_xlabel('Symbol', fontsize=12, fontweight='bold')
    ax.set_ylabel('7-Day Volatility', fontsize=12, fontweight='bold')
    ax.set_title('Volatility Distribution by Cryptocurrency Symbol', 
                 fontsize=14, fontweight='bold', pad=20)
    plt.suptitle('')  # Убрать автоматический title от boxplot
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
    
    # Инсайты
    print("\n📊 ИНСАЙТЫ ПО ВОЛАТИЛЬНОСТИ:")
    volatility_stats = market_overview.groupby('symbol')['volatility_7d'].agg(['mean', 'std', 'max'])
    volatility_stats = volatility_stats.sort_values('mean', ascending=False)
    print(volatility_stats)
    
    most_volatile = volatility_stats.index[0]
    least_volatile = volatility_stats.index[-1]
    print(f"\n   - Самая волатильная: {most_volatile} (средняя волатильность: {volatility_stats.loc[most_volatile, 'mean']:.4f})")
    print(f"   - Наиболее стабильная: {least_volatile} (средняя волатильность: {volatility_stats.loc[least_volatile, 'mean']:.4f})")
    print(f"   - Рекомендация: {least_volatile} подходит для консервативных инвесторов")
else:
    print("⚠️ Нет данных для анализа волатильности")

## 6. Визуализация 3: Sentiment Analysis новостей

**Инсайт**: Динамика sentiment новостей помогает понять общий настрой рынка и может служить опережающим индикатором движения цен.

In [ ]:
if len(news_impact) > 0:
    # Подготовка данных
    news_impact_sorted = news_impact.sort_values('date')
    
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10))
    
    # График 1: Количество новостей по sentiment
    ax1.bar(news_impact_sorted['date'], news_impact_sorted['positive_news'], 
            label='Positive', alpha=0.7, color='green')
    ax1.bar(news_impact_sorted['date'], news_impact_sorted['negative_news'], 
            label='Negative', alpha=0.7, color='red', 
            bottom=news_impact_sorted['positive_news'])
    ax1.bar(news_impact_sorted['date'], news_impact_sorted['neutral_news'], 
            label='Neutral', alpha=0.7, color='gray',
            bottom=news_impact_sorted['positive_news'] + news_impact_sorted['negative_news'])
    
    ax1.set_xlabel('Date', fontsize=11, fontweight='bold')
    ax1.set_ylabel('Number of News', fontsize=11, fontweight='bold')
    ax1.set_title('News Sentiment Distribution Over Time', fontsize=13, fontweight='bold')
    ax1.legend(loc='upper left')
    ax1.grid(True, alpha=0.3, axis='y')
    plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)
    
    # График 2: Средний sentiment score vs изменение цены
    ax2_twin = ax2.twinx()
    
    line1 = ax2.plot(news_impact_sorted['date'], news_impact_sorted['avg_sentiment_score'],
                     label='Avg Sentiment Score', color='blue', linewidth=2, marker='o')
    line2 = ax2_twin.plot(news_impact_sorted['date'], news_impact_sorted['avg_price_change_pct'],
                          label='Avg Price Change %', color='orange', linewidth=2, marker='s')
    
    ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
    ax2.set_ylabel('Sentiment Score', fontsize=11, fontweight='bold', color='blue')
    ax2_twin.set_ylabel('Price Change %', fontsize=11, fontweight='bold', color='orange')
    ax2.set_title('Sentiment Score vs Price Change Correlation', fontsize=13, fontweight='bold')
    ax2.tick_params(axis='y', labelcolor='blue')
    ax2_twin.tick_params(axis='y', labelcolor='orange')
    ax2.grid(True, alpha=0.3)
    plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
    
    # Объединенная легенда
    lines = line1 + line2
    labels = [l.get_label() for l in lines]
    ax2.legend(lines, labels, loc='upper left')
    
    plt.tight_layout()
    plt.show()
    
    # Инсайты
    print("\n📊 ИНСАЙТЫ ПО НОВОСТЯМ:")
    total_news = news_impact['total_news'].sum()
    total_positive = news_impact['positive_news'].sum()
    total_negative = news_impact['negative_news'].sum()
    total_neutral = news_impact['neutral_news'].sum()
    
    print(f"   - Всего новостей проанализировано: {total_news}")
    print(f"   - Positive: {total_positive} ({total_positive/total_news*100:.1f}%)")
    print(f"   - Negative: {total_negative} ({total_negative/total_news*100:.1f}%)")
    print(f"   - Neutral: {total_neutral} ({total_neutral/total_news*100:.1f}%)")
    
    # Корреляция
    positive_corr = news_impact['sentiment_price_correlation'].value_counts().get('positive_correlation', 0)
    negative_corr = news_impact['sentiment_price_correlation'].value_counts().get('negative_correlation', 0)
    no_corr = news_impact['sentiment_price_correlation'].value_counts().get('no_correlation', 0)
    
    total_days = len(news_impact)
    print(f"\n   - Дней с положительной корреляцией: {positive_corr} ({positive_corr/total_days*100:.1f}%)")
    print(f"   - Дней с отрицательной корреляцией: {negative_corr} ({negative_corr/total_days*100:.1f}%)")
    print(f"   - Дней без корреляции: {no_corr} ({no_corr/total_days*100:.1f}%)")
    
    if positive_corr > negative_corr:
        print("\n   ✅ Новости преимущественно влияют на цены в ожидаемом направлении")
    else:
        print("\n   ⚠️ Рынок часто игнорирует или реагирует противоположно новостному фону")
else:
    print("⚠️ Нет данных для анализа новостей")

## 7. Визуализация 4: Candlestick Chart (OHLC)

**Инсайт**: Классический свечной график показывает детальную картину торгов с ценами открытия, закрытия, максимумами и минимумами.

In [ ]:
if len(daily_agg) > 0:
    # Фильтр для BTCUSDT за последние 30 дней
    btc_daily = daily_agg[daily_agg['symbol'] == 'BTCUSDT'].sort_values('date').tail(30)
    
    if len(btc_daily) > 0:
        fig, ax = plt.subplots(figsize=(16, 8))
        
        # Подготовка данных для candlestick
        up = btc_daily[btc_daily['close_price'] >= btc_daily['open_price']]
        down = btc_daily[btc_daily['close_price'] < btc_daily['open_price']]
        
        # Зеленые свечи (рост)
        ax.bar(up.index, up['close_price'] - up['open_price'], 
               bottom=up['open_price'], color='green', alpha=0.8, width=0.6)
        ax.bar(up.index, up['high_price'] - up['close_price'], 
               bottom=up['close_price'], color='green', alpha=0.4, width=0.1)
        ax.bar(up.index, up['open_price'] - up['low_price'], 
               bottom=up['low_price'], color='green', alpha=0.4, width=0.1)
        
        # Красные свечи (падение)
        ax.bar(down.index, down['open_price'] - down['close_price'], 
               bottom=down['close_price'], color='red', alpha=0.8, width=0.6)
        ax.bar(down.index, down['high_price'] - down['open_price'], 
               bottom=down['open_price'], color='red', alpha=0.4, width=0.1)
        ax.bar(down.index, down['close_price'] - down['low_price'], 
               bottom=down['low_price'], color='red', alpha=0.4, width=0.1)
        
        # Настройка осей
        ax.set_xticks(range(len(btc_daily)))
        ax.set_xticklabels([d.strftime('%Y-%m-%d') for d in btc_daily['date']], rotation=45)
        ax.set_xlabel('Date', fontsize=12, fontweight='bold')
        ax.set_ylabel('Price (USDT)', fontsize=12, fontweight='bold')
        ax.set_title('BTC/USDT Candlestick Chart (Last 30 Days)', 
                     fontsize=14, fontweight='bold', pad=20)
        ax.grid(True, alpha=0.3, axis='y')
        
        plt.tight_layout()
        plt.show()
        
        # Инсайты
        print("\n📊 ИНСАЙТЫ ПО ТОРГОВЛЕ:")
        print(f"   - Средняя дневная волатильность: {btc_daily['price_change_pct'].abs().mean():.2f}%")
        print(f"   - Максимальный рост за день: {btc_daily['price_change_pct'].max():.2f}%")
        print(f"   - Максимальное падение за день: {btc_daily['price_change_pct'].min():.2f}%")
        print(f"   - Зеленых дней (рост): {len(up)} из {len(btc_daily)} ({len(up)/len(btc_daily)*100:.1f}%)")
        print(f"   - Красных дней (падение): {len(down)} из {len(btc_daily)} ({len(down)/len(btc_daily)*100:.1f}%)")
    else:
        print("⚠️ Недостаточно данных BTCUSDT для candlestick chart")
else:
    print("⚠️ Нет данных для candlestick chart")

## 8. Визуализация 5: Корреляционная матрица

**Инсайт**: Тепловая карта показывает взаимосвязь между различными метриками рынка (цена, объем, волатильность, sentiment).

In [ ]:
if len(market_overview) > 0 and len(news_impact) > 0:
    # Объединение данных по датам для корреляционного анализа
    btc_market = market_overview[market_overview['symbol'] == 'BTCUSDT'][['date', 'current_price', 'volatility_7d', 'total_volume']]
    merged_data = pd.merge(btc_market, news_impact[['date', 'avg_sentiment_score', 'total_news']], 
                          on='date', how='inner')
    
    if len(merged_data) > 0:
        # Вычисление корреляций
        correlation_matrix = merged_data[[
            'current_price', 'volatility_7d', 'total_volume', 
            'avg_sentiment_score', 'total_news'
        ]].corr()
        
        # Визуализация
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
                   center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8}, ax=ax)
        
        ax.set_title('Correlation Matrix: Market Metrics vs News Sentiment', 
                    fontsize=14, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.show()
        
        # Инсайты
        print("\n📊 КЛЮЧЕВЫЕ КОРРЕЛЯЦИИ:")
        
        # Найти сильные корреляции
        for i in range(len(correlation_matrix)):
            for j in range(i+1, len(correlation_matrix)):
                corr_value = correlation_matrix.iloc[i, j]
                if abs(corr_value) > 0.5:
                    col1 = correlation_matrix.index[i]
                    col2 = correlation_matrix.columns[j]
                    direction = "положительная" if corr_value > 0 else "отрицательная"
                    strength = "сильная" if abs(corr_value) > 0.7 else "умеренная"
                    print(f"   - {col1} ↔ {col2}: {strength} {direction} корреляция ({corr_value:.3f})")
        
        # Специфичные выводы
        if 'avg_sentiment_score' in correlation_matrix and 'current_price' in correlation_matrix:
            sentiment_price_corr = correlation_matrix.loc['avg_sentiment_score', 'current_price']
            if abs(sentiment_price_corr) > 0.3:
                print(f"\n   ✅ Sentiment новостей имеет заметное влияние на цену (корреляция: {sentiment_price_corr:.3f})")
            else:
                print(f"\n   ⚠️ Sentiment новостей слабо коррелирует с ценой (корреляция: {sentiment_price_corr:.3f})")
    else:
        print("⚠️ Недостаточно общих данных для корреляционного анализа")
else:
    print("⚠️ Нет данных для корреляционного анализа")

## 9. Визуализация 6: Объем торгов

**Инсайт**: Анализ торгового объема помогает подтвердить или опровергнуть ценовые движения.

In [ ]:
if len(daily_agg) > 0:
    btc_volume = daily_agg[daily_agg['symbol'] == 'BTCUSDT'].sort_values('date')
    
    if len(btc_volume) > 0:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(16, 10), sharex=True)
        
        # График 1: Цена
        ax1.plot(btc_volume['date'], btc_volume['close_price'], 
                color='blue', linewidth=2, label='Close Price')
        ax1.set_ylabel('Price (USDT)', fontsize=11, fontweight='bold')
        ax1.set_title('BTC/USDT Price and Trading Volume Analysis', 
                     fontsize=14, fontweight='bold', pad=20)
        ax1.legend(loc='upper left')
        ax1.grid(True, alpha=0.3)
        
        # График 2: Объем торгов
        colors = ['green' if pc >= 0 else 'red' for pc in btc_volume['price_change_pct']]
        ax2.bar(btc_volume['date'], btc_volume['total_volume'], 
               color=colors, alpha=0.7)
        ax2.set_xlabel('Date', fontsize=11, fontweight='bold')
        ax2.set_ylabel('Volume', fontsize=11, fontweight='bold')
        ax2.set_title('Trading Volume (Green = Price Up, Red = Price Down)', 
                     fontsize=12, fontweight='bold')
        ax2.grid(True, alpha=0.3, axis='y')
        plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45)
        
        plt.tight_layout()
        plt.show()
        
        # Инсайты
        print("\n📊 ИНСАЙТЫ ПО ОБЪЕМУ:")
        avg_volume = btc_volume['total_volume'].mean()
        max_volume = btc_volume['total_volume'].max()
        max_volume_date = btc_volume[btc_volume['total_volume'] == max_volume]['date'].iloc[0]
        
        print(f"   - Средний дневной объем: {avg_volume:,.0f}")
        print(f"   - Максимальный объем: {max_volume:,.0f} ({max_volume_date})")
        
        # Корреляция объема и изменения цены
        volume_price_corr = btc_volume[['total_volume', 'price_change_pct']].corr().iloc[0, 1]
        print(f"   - Корреляция объема и изменения цены: {volume_price_corr:.3f}")
        
        if abs(volume_price_corr) > 0.3:
            print("   ✅ Объем подтверждает ценовые движения")
        else:
            print("   ⚠️ Объем слабо связан с движением цены")
    else:
        print("⚠️ Нет данных BTCUSDT для анализа объемов")
else:
    print("⚠️ Нет данных для анализа объемов")

## 10. Общие выводы и рекомендации

In [ ]:
print("="*80)
print("ИТОГОВЫЕ ВЫВОДЫ И РЕКОМЕНДАЦИИ")
print("="*80)

print("\n1️⃣ АНАЛИЗ ТРЕНДОВ:")
if len(market_overview) > 0:
    trend_distribution = market_overview['trend_indicator'].value_counts()
    dominant_trend = trend_distribution.index[0]
    print(f"   - Доминирующий тренд на рынке: {dominant_trend}")
    print(f"   - Распределение: {dict(trend_distribution)}")

print("\n2️⃣ ВОЛАТИЛЬНОСТЬ И РИСКИ:")
if len(market_overview) > 0:
    avg_volatility = market_overview.groupby('symbol')['volatility_7d'].mean().mean()
    print(f"   - Средняя волатильность по всем активам: {avg_volatility:.4f}")
    if avg_volatility > 0.05:
        print("   ⚠️ ВЫСОКАЯ волатильность - рекомендуется осторожность")
    else:
        print("   ✅ Умеренная волатильность - благоприятные условия")

print("\n3️⃣ ВЛИЯНИЕ НОВОСТЕЙ:")
if len(news_impact) > 0:
    positive_days = (news_impact['sentiment_price_correlation'] == 'positive_correlation').sum()
    total_days = len(news_impact)
    print(f"   - Новости коррелируют с ценой в {positive_days}/{total_days} дней ({positive_days/total_days*100:.1f}%)")
    
    if positive_days / total_days > 0.6:
        print("   ✅ Новости - надежный индикатор движения цен")
    else:
        print("   ⚠️ Рынок часто игнорирует новостной фон")

print("\n4️⃣ ТОРГОВЫЕ РЕКОМЕНДАЦИИ:")
print("   - Используйте MA(7) и MA(30) для определения точек входа/выхода")
print("   - Мониторьте sentiment новостей как дополнительный сигнал")
print("   - Избегайте торговли в периоды экстремальной волатильности")
print("   - Подтверждайте ценовые движения анализом объемов")

print("\n5️⃣ ИСТОЧНИКИ ДАННЫХ:")
print("   - Binance API: исторические OHLCV данные")
print("   - Crypto.news RSS: новости с sentiment analysis")
print("   - DBT трансформации: STG → ODS → DM")
print("   - Elementary: мониторинг качества данных")

print("\n" + "="*80)
print("Анализ завершен. Данные актуальны на:", datetime.now().strftime('%Y-%m-%d %H:%M:%S'))
print("="*80)

## Заключение

Данный анализ демонстрирует полный цикл работы с данными:
1. **Сбор данных** - Binance API + Crypto.news RSS
2. **Хранение** - MongoDB (временное) → PostgreSQL (DWH)
3. **Трансформации** - DBT (STG → ODS → DM)
4. **Анализ** - Python + Pandas + Matplotlib + Seaborn
5. **Инсайты** - Визуализации с бизнес-выводами

Все данные прошли валидацию через Elementary тесты и готовы к использованию в production.

In [ ]:
# Закрытие подключения
conn.close()
print("✅ Подключение к базе данных закрыто")